## Hyper-parameter optimization in concise

Hyper-parameter optimization comes quite handy in deep learning. There are different architectures to try out, different parameters, and we only have a limited amount of time to inspect the results of a particular modeling choice and decide on the next set of hyper-parmaeters to try out.

By hyper-parameters I refer to any parameter that are not optimized with the learning algorithm (i.e. gradient descent). Some examples are: learning rate of gradient descent, batch size, number of hidden layers, whether to use skip-connections, particular way of pre-processing the data (say data augmentation amount), etc etc.

In genomics, being able to explore a vast amount of modeling choices automatically is important. Only a handful of papers (if any) have been written for your given task at hand, hence the space of different modeling choices has not been extensively explored, as for say image classification on [ImageNet](http://www.image-net.org/).

## Prerequisite

- please go throught the hyperopt wiki: https://github.com/hyperopt/hyperopt/wiki

## Note

- hyperopt doesn't work nicely with ipython. Run the examples in normal python shell

## Steps in hyper-parameter optimization

1. Parametrize your preprocessing, model and training procedure
2. Define a range of values (and their distribution) for all the parameters
3. Run the optimization in a distributed fashion (on a cluster/multiple machines in parallel)

## 1. Parametrize your preprocessing, model and training procedure

Hyper-parameter optimization in CONCISE assumes you have defined two functions: `data(...)` and `model(...)`.

### data() function

First, the define `data(...)`: a function returing the whole trainig, validation and test dataset.

- data() - returns a tuple: (train, test) or (train, valid, test), where:
  - train = (X_train, y_train, other1, other2, ...)
  - valid = (X_valid, y_valid)
  - test = (X_test, y_test)

- `X_*` is a numpy array, list of numpy arrays or a dictionary of numpy array
- `y_*` is the response variable

`X_train` and `y_train` are feeded to the model fitting method call: `model.fit(x=X_train, y = y_train, ...)`

#### Example



In [96]:
from concise.preprocessing import encodeDNA
import pandas as pd
import numpy as np

In [98]:
from joblib import Memory
mem = Memory(cachedir='/tmp/joblib')
# TODO - make the memoization work

In [99]:
#@mem.cache
def data(seq_length=101):
      
    def load(split="train"):
        dt = pd.read_csv("../data/RBP/PUM2_{0}.csv".format(split))
        # DNA/RNA sequence
        xseq = encodeDNA(dt.seq, maxlen=seq_length, seq_align='center')
        # response variable
        y = dt.binding_site.as_matrix().reshape((-1, 1)).astype("float")
        if split=="train":
            from concise.data import attract
            # add also the pwm_list
            pwm_list = attract.get_pwm_list(["129"])
            return {"seq": xseq}, y, pwm_list
        else:
            return {"seq": xseq}, y

    return load("train"), load("valid"), load("test")

In [100]:
train, valid, test = data()

### model() function

`model()` returns a compiled Keras model. The only restriction on the parameters is that the function needs a `train_data` parameter. This allows you to extract the shape of your dataset.

In [101]:
import concise.layers as cl
import keras.layers as kl
import concise.initializers as ci
import concise.regularizers as cr
from keras.callbacks import EarlyStopping
from keras.models import Model, load_model
from keras.optimizers import Adam

def model(train_data, filters=1, kernel_size=9, motif_init=None, lr=0.001):
    seq_length = train_data[0]["seq"].shape[1]
    pwm_list = train_data[2]
    
    if motif_init is not None:
        # Motif init is a dictionary with fields: "stddev" 
        kinit = ci.PSSMKernelInitializer(pwm_list, 
                                         stddev=motif_init.get("stddev", 0.05),  # if not specified, use 0.05
                                         add_noise_before_Pwm2Pssm=True)
        binit = "zeros"
    else:
        kinit = "glorot_uniform"
        binit = "zeros"
        
        
    # sequence
    in_dna = cl.InputDNA(seq_length=seq_length, name="seq")
    x = cl.ConvDNA(filters=filters, 
                   kernel_size=kernel_size, 
                   activation="relu",
                   kernel_initializer=kinit,
                   bias_initializer=binit,
                   name="conv1")(in_dna)
    x = kl.AveragePooling1D(pool_size=4)(x)
    x = kl.Flatten()(x)
    
    x = kl.Dense(units=1)(x)
    m = Model(in_dna, x)
    m.compile(Adam(lr=lr), loss="binary_crossentropy", metrics=["acc"])
    return m

## 2. Define a range of values (and their distribution) for all the parameters

Currently, our hyper-parameters are:
- data:
  - seq_length
- model:
  - filters
  - kernel_size
  - motif_init (yes or no)
    - if yes: stddev
  - lr

Moreover, in the training procedure we could use different batch_sizes or different callback parameters.

Let's define a hyper-opt pyll graph defining the ranges of parameters:

In [102]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

hyper_params = {
    "data": {
        "seq_length": hp.choice("d_seq_length", (101, 51, 21))
    },
    "model": {
        "filters": hp.choice("m_filters", (1, 16)),
        "kernel_size": 15,
        "motif_init": hp.choice("m_motif_init", (
                None,
                {"stddev": hp.uniform("m_stddev", 0, 0.2)}
            )),
        "lr": hp.loguniform("m_lr", np.log(1e-4), np.log(1e-2)) # 0.0001 - 0.01
        },
    "fit": {
        "epochs": 1,
        "patience": 5,
        "batch_size": 128,
    }
}

This dictionary defines the parameter search space. Specifically, we are saying that
- `seq_length` in the data function should be 21, 51 or 101
- we should use 1 or 16 filter
- the learning rate should be from the range `[1e-4, 1e-2]`, sampled on a log-scale
- ...

This allows us to also specify nested parameters, as shown for `["model"]["motif_init"]`. We want to try both cases: with and without motif initialization and in the case of motif initialization, we want to explore different options for the "sttdev" parameter.

Read the `hyperopt` documentation about [defining the search space](https://github.com/hyperopt/hyperopt/wiki/FMin#2-defining-a-search-space) for more information.

## 3. Run the optimization

In [103]:
from concise.hyopt import CompileFN, CMongoTrials, test_fn

To run hyper-opt, you need to define the function to minimize and the parameters you want to explore. Example:

In [71]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(x):
    return {'loss': x ** 2, 'status': STATUS_OK }

best = fmin(objective,
    space=hp.uniform('x', -10, 10),
    algo=tpe.suggest,
    max_evals=100)

print(best)

2017-08-14 14:34:14,780 [INFO] tpe_transform took 0.000574 seconds
2017-08-14 14:34:14,781 [INFO] TPE using 0 trials
2017-08-14 14:34:14,783 [INFO] tpe_transform took 0.000595 seconds
2017-08-14 14:34:14,784 [INFO] TPE using 1/1 trials with best loss 14.293453
2017-08-14 14:34:14,785 [INFO] tpe_transform took 0.000556 seconds
2017-08-14 14:34:14,786 [INFO] TPE using 2/2 trials with best loss 14.293453
2017-08-14 14:34:14,788 [INFO] tpe_transform took 0.000517 seconds
2017-08-14 14:34:14,789 [INFO] TPE using 3/3 trials with best loss 14.293453
2017-08-14 14:34:14,791 [INFO] tpe_transform took 0.000526 seconds
2017-08-14 14:34:14,793 [INFO] TPE using 4/4 trials with best loss 2.047095
2017-08-14 14:34:14,796 [INFO] tpe_transform took 0.000939 seconds
2017-08-14 14:34:14,798 [INFO] TPE using 5/5 trials with best loss 2.047095
2017-08-14 14:34:14,801 [INFO] tpe_transform took 0.000642 seconds
2017-08-14 14:34:14,802 [INFO] TPE using 6/6 trials with best loss 2.047095
2017-08-14 14:34:14,80

{'x': 0.00361393118788067}


For the case of supervised learning, our objective function is the evaluation error. Concise provides a class to compile the objective function given the data() and the model() function:

In [112]:
import hyopt_example
objective = CompileFN(db_name="mydb", exp_name="motif_initialization",  # experiment name
                      data_fn=hyopt_example.data,
                      model_fn=hyopt_example.model, 
                      add_eval_metrics=["auprc", "auc"], # metrics from concise.eval_metrics, you can also use your own
                      loss_metric="auprc", # which metric to optimize for
                      loss_metric_mode="max", # maximum should be searched for
                      valid_split=None, # use valid from the data function
                      save_model='best', # checkpoint the best model
                      save_results=True, # save the results as .json (in addition to mongoDB)
                      save_dir="./hyperopt/")  # place to store the models

Now, we can use hyper-opt do search the hyper-parameters:

### Test if everything works

Before starting many workers, its good to try running the objective function once. It's much easier to debug it that way.

In [119]:
# Test if the objective function works on a subset of data for one random hyper-parameter set
test_fn(objective, hyper_params)
# Doesn't work in ipython notebook

### Run locally and sequentially

Trials() from hyperopt will run the jobs sequentially on a single machine

In [117]:
trials = Trials("motif_initialization")
best = fmin(objective,
            space=hyper_params,
            algo=tpe.suggest,
            trials=trials,
            max_evals=3)

2017-08-14 14:52:46,515 [INFO] tpe_transform took 0.002803 seconds
2017-08-14 14:52:46,517 [INFO] TPE using 0 trials
2017-08-14 14:52:46,520 [INFO] Load data...
2017-08-14 14:52:56,455 [INFO] job exception: No module named 'winreg'


['loss', 'acc']


ImportError: No module named 'winreg'

### Running the optimization in parallel on multiple workers

MongoTrials will defer the work to multiple workers running in parallel. All the results will get stored to the MongoDB database.

In [120]:
# just replace Trials with CMongoTrials
trials = CMongoTrials(db_name="mydb", exp_name="motif_initialization")
best = fmin(objective,
            space=hyper_params,
            algo=tpe.suggest,
            trials=trials,
            max_evals=3)

2017-08-14 14:55:37,909 [INFO] PROTOCOL mongo
2017-08-14 14:55:37,910 [INFO] USERNAME None
2017-08-14 14:55:37,911 [INFO] HOSTNAME ouga03
2017-08-14 14:55:37,911 [INFO] PORT 1234
2017-08-14 14:55:37,912 [INFO] PATH /mydb/jobs
2017-08-14 14:55:37,913 [INFO] DB mydb
2017-08-14 14:55:37,914 [INFO] COLLECTION jobs
2017-08-14 14:55:37,915 [INFO] PASS None


ServerSelectionTimeoutError: ouga03:1234: [Errno 111] Connection refused

This script waits for workers to complete the work. Start the workers by running the following bash script many times in parallel:

```bash
#!/bin/bash
#
# Slurm parameters
#SBATCH --mem=16G
#SBATCH -c 4
# ------------------

DB=Concise__Splice_branchpoints
# path to the repository containing the script
export PYTHONPATH=~/projects-work/deepcis/Scripts/Concise/Splice_branchpoints

hyperopt-mongo-worker \
    --mongo=ouga03:1234/$DB \
    --poll-interval=1 \
    --reserve-timeout=3600
```

## Code organization

- data.py
  - contains data() function
- model.py
  - contains model() function
- train.py
  - Defines the hyper_parameter ranges
  - Runs hyperopt.fmin()  

#### Examples
See data.py, model.py and train.py in: 
- https://i12g-gagneurweb.informatik.tu-muenchen.de/gitlab/avsec/deepcis/tree/master/Scripts/RBP/Eclip/predictive_models
- https://i12g-gagneurweb.informatik.tu-muenchen.de/gitlab/avsec/deepcis/blob/master/Scripts/Concise/Splice_branchpoints/